<a href="https://colab.research.google.com/github/Narabcs/doctorship/blob/main/fcp_nara.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd

!git clone https://github.com/Narabcs/doctorship/

fatal: destination path 'doctorship' already exists and is not an empty directory.


### Etapa 01:

Calculando os valores de tortuosidade utilizando os dados da água
---

In [6]:
dados = pd.read_csv('/content/doctorship/parameters_water.csv')
dados['y_ress'] = dados['yexp']/dados['xexp']
dados

,yexp,mi,xexp,pf,meio,y_ress
0,31500000.0,0.000891,0.00800,997.05,50,3.937500e+09
1,63000000.0,0.000891,0.01250,997.05,50,5.040000e+09
2,94500000.0,0.000891,0.01480,997.05,50,6.385135e+09
3,126000000.0,0.000891,0.01730,997.05,50,7.283237e+09
4,157000000.0,0.000891,0.01930,997.05,50,8.134715e+09
5,189000000.0,0.000891,0.02060,997.05,50,9.174757e+09
6,220000000.0,0.000891,0.02240,997.05,50,9.821429e+09
7,15700000.0,0.000891,0.00727,997.05,120,2.159560e+09
8,31500000.0,0.000891,0.01060,997.05,120,2.971698e+09
9,63000000.0,0.000891,0.01460,997.05,120,4.315068e+09


In [ ]:
from scipy import stats

ress_50 = stats.linregress(x=dados.query('meio==50')['xexp'],y=dados.query('meio==50')['y_ress'])
ress_120 = stats.linregress(x=dados.query('meio==120')['xexp'],y=dados.query('meio==120')['y_ress'])

results = {'meio=50':{'r_squared':ress_50.rvalue,
           'intercept': ress_50.intercept,
           'slope': ress_50.slope},
           'meio=120':{'r_squared':ress_120.rvalue,
           'intercept': ress_120.intercept,
           'slope': ress_120.slope}
           }

results = pd.DataFrame(results)
results

In [ ]:
def tortuosity(slope,k,pf):
  c = slope*k**0.5/pf

  return c

In [ ]:
c_50 = tortuosity(results['meio=50'].slope,1.48e-11,dados['pf'].unique())
c_120 = tortuosity(results['meio=120'].slope,3.95e-11,dados['pf'].unique())
print(f'c_50 = {float(c_50)}, c_120 = {float(c_120)}')

### Etapa 02: 

Estimando os valores de K para as equações de Forchheimmer e Darcy.

In [ ]:
dados_estimacao = pd.read_excel('parametros.xlsx',sheet_name='OnlyXG')
dados_estimacao

In [ ]:
def forch_opt(x, dados):
  forch = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/x[0]**0.5)**(dados[2]-1))/x[0] * dados[3] + dados[6]*dados[5]/x[0]**0.5 * dados[3]**2
  err_rel = abs(dados[0] - forch) / dados[0] 

  return err_rel

def darcy_opt(k, dados):
  darcy = (dados[1] * ((1.1/(2.5*dados[4])**0.5) * dados[3]/k**0.5)**(dados[2]-1))/k * dados[3]
  err_rel = abs(dados[0] - darcy) / dados[0] 
  return err_rel

def modelo(dados, model:str='forch'):
  
  if model == 'forch':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_forch']**0.5)**(dados['n']-1))
    y = mu/dados['k_forch'] * dados['xexp'] + dados['c']*dados['pf']/dados['k_forch']**0.5 * dados['xexp']**2

  if model == 'darcy':
    mu = (dados['m'] * ((1.1/(2.5*dados['es'])**0.5) * dados['xexp']/dados['k_darcy']**0.5)**(dados['n']-1))
    y = mu/dados['k_darcy'] * dados['xexp']

  return y, mu

In [ ]:
from scipy.optimize import differential_evolution as de

params_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=forch_opt,bounds=params_bound,args=(dados_selecionados,))
  results.append(value)

df_forch = pd.DataFrame(results)

k_bound = [(1e-20,1e-10)]

results = []

for row in dados_estimacao.iterrows():
  dados_selecionados = [row[1]['yexp'], row[1]['m'], row[1]['n'], row[1]['xexp'], row[1]['es'], row[1]['pf'], row[1]['c']]
  value = de(func=darcy_opt,bounds=k_bound,args=(dados_selecionados,))
  results.append(value)

df_darcy = pd.DataFrame(results)

In [ ]:
k_forch = []
k_darcy = []

for item in zip(df_forch.x,df_darcy.x):
  k_forch.append(item[0][0])
  k_darcy.append(item[1][0])
dados_estimacao['k_forch'] = k_forch
dados_estimacao['k_darcy'] = k_darcy
dados_estimacao['dpl_forch'], dados_estimacao['mu_forch'] = modelo(dados_estimacao,model='forch')
dados_estimacao['dpl_darcy'], dados_estimacao['mu_darcy'] = modelo(dados_estimacao,model='darcy')
dados_estimacao

### Etapa 03:

Estimar a permeabilidade da torta

In [ ]:
def modelo_torta(kc,dados):

  # km = x[0]
  # kc = x[1]

  ps = dados[0]
  ec = dados[1]
  m = dados[2]
  n = dados[3]
  rho_fil = dados[4]
  const_c = dados[5]
  dP = dados[6]
  rm = dados[7]
  A = dados[8]
  V = dados[9]
  V0 = dados[10]
  dt_real = dados[11]

  # eqdarcy (dpl,mu)

  alpha = 1 / (ps*(1-ec)*kc)
  param_a = m*(1.1/(2.5*ec)**0.5 * 1/ kc**0.5)**(n-1)
  param_b = param_a*alpha*const_c*rho_fil / (A**(n+1)*dP)
  param_c = param_a * rm / (A**n * dP)
  param_d = 1/n

  dt_cal = ((param_c+V*param_b)**(param_d+1) - (param_c+V0*param_b)**(param_d+1)) / (param_b*(param_d+1))

  erro_rel = abs(dt_real-dt_cal) / dt_real

  return erro_rel

In [ ]:
dados_torta = pd.read_excel('parametros.xlsx',sheet_name='Test')
dados_torta['dt_real'] = dados_torta['t']-dados_torta['t0']
dados_torta

In [ ]:
params_bound = [(1e-20,1e-10)]
# params_bound = [(1e-20,1e-10),(1,e-20,1e-10)]

results = []

for row in dados_torta.iterrows():
  dados_selecionados = [row[1]['ps'], 
                        row[1]['ec'], 
                        row[1]['m'], 
                        row[1]['n'], 
                        row[1]['rho_fil'], 
                        row[1]['Cs'],
                        row[1]['yexp'],
                        row[1]['Rm'],
                        row[1]['A'],
                        row[1]['V'],
                        row[1]['V0'],
                        row[1]['dt_real']]

  value = de(func=modelo_torta,bounds=params_bound,args=(dados_selecionados,))
  results.append(value)

df_torta = pd.DataFrame(results)
df_torta

In [ ]:
k_torta = []

for item in df_torta.x:
  k_torta.append(float(item))
  
dados_torta['k_torta'] = k_torta
dados_torta